# Session 9: Data Wrangling and Reverse Geocoding the Rental Data

Last session we began using Pandas to clean up our rental data using pandas methods. This is where we left off:

In [1]:
# import libraries and read in the csv file
import re as re, pandas as pd, numpy as np, requests, json
df = pd.read_csv('data/bay.csv')

# clean price and neighborhood
df.price = df.price.str.strip('$').astype('float64')
df.neighborhood = df.neighborhood.str.strip().str.strip('(').str.strip(')')

# break out the date into month day year columns
df['month'] = df['date'].str.split().str[0]
df['day'] = df['date'].str.split().str[1].astype('int32')
df['year'] = df['date'].str.split().str[2].astype('int32')
del df['date']

df[:10]

,neighborhood,title,price,bedrooms,pid,longitude,subregion,link,latitude,sqft,month,day,year
0,bayview,Take A TOUR ON OUR ONE FURNISHED BEDROOM TODAY,950,/ 1br -,4076905111,-122.396965,SF,/sfc/apa/4076905111.html,37.761216,/ 1br -,Sep,18,2013
1,bayview,Only walking distance to major shopping centers.,950,/ 1br -,4076901755,-122.396793,SF,/sfc/apa/4076901755.html,37.761080,/ 1br -,Sep,18,2013
2,bayview,"furnished - 1 Bedroom(s), 1 Bath(s), Air Condi...",950,/ 1br -,4076899340,-122.397100,SF,/sfc/apa/4076899340.html,37.762100,/ 1br -,Sep,18,2013
3,financial district,"*NEW* Beautiful, Upscale Condo in Historic Jac...",3300,/ 1br - 830ft² -,4067393707,-122.399747,SF,/sfc/apa/4067393707.html,37.798108,/ 1br - 830ft² -,Sep,18,2013
4,visitacion valley,楼上全层3房,2000,/ 3br - 1280ft² -,4076901071,NaN,SF,/sfc/apa/4076901071.html,NaN,/ 3br - 1280ft² -,Sep,18,2013
5,bernal heights,"2 Bedroom House w/Bonus Room, Fireplace, Yard,...",3850,/ 2br -,4076899828,NaN,SF,/sfc/apa/4076899828.html,NaN,/ 2br -,Sep,18,2013
6,marina / cow hollow,"♥♥♥ Chic, cozy & sunny in Marina ♥♥♥",3399,,4076892173,NaN,SF,/sfc/apa/4076892173.html,NaN,,Sep,18,2013
7,north beach / telegraph hill,Fully Furnished 2BR/2BA Condo with Bay View,5250,/ 2br - 1000ft² -,4076895917,NaN,SF,/sfc/apa/4076895917.html,NaN,/ 2br - 1000ft² -,Sep,18,2013
8,bayview,"We accept either 6, 12 month or month-to-month...",950,/ 1br -,4076896866,-122.397137,SF,/sfc/apa/4076896866.html,37.762030,/ 1br -,Sep,18,2013
9,bayview,"3 Bayside Village Pl, San Francisco, CA 94107",950,/ 1br -,4076892885,-122.396450,SF,/sfc/apa/4076892885.html,37.764065,/ 1br -,Sep,18,2013


Now we need to use a couple of more advanced methods to clean up bedrooms and sqft.  Let's start with sqft.

For this, at least one way to do this is to grab the starting and ending values of the substring we want to extract, keying on the '-' and the 'br' paers of the sqft text field to grab the characters in between.  The Pandas tutorial showed one way to iterate across a DataFrame. Here is another.  Notice that we also have to insert some logic to handle the case where sqft is not given.

In [2]:
# clean up the sqft using an iterator over the rows in the DataFrame
for i in df.sqft.index:
    if('ft' in df.sqft[i]):
        end = df.sqft[i].find('ft')
        begin = df.sqft[i].find('- ') + 2
        if(begin > end):
            begin = df.sqft[i].find('/ ') + 2
        df.sqft[i] = df.sqft[i][begin:end]
    else:
        df.sqft[i] = np.nan
df.sqft = df.sqft.astype('float64')

df['sqft'][:10]

0     NaN
1     NaN
2     NaN
3     830
4    1280
5     NaN
6     NaN
7    1000
8     NaN
9     NaN
Name: sqft, dtype: float64

You can use the same approach to extract the bedrooms.  

In [3]:
for i in df.bedrooms.index:
    if('br' in df.bedrooms[i]):
        end = df.bedrooms[i].find('br')
        begin = df.bedrooms[i].find('/ ') + 1
        if(begin > end):
            begin = df.bedrooms[i].find('/ ') + 1
        df.bedrooms[i] = df.bedrooms[i][begin:end]
    else:
        df.bedrooms[i] = np.nan
df.bedrooms = df.bedrooms.astype('float64')

df['bedrooms'][:10]

0     1
1     1
2     1
3     1
4     3
5     2
6   NaN
7     2
8     1
9     1
Name: bedrooms, dtype: float64

Now a more advanced topic: using regular expressions (regex) which is available in a Python library 're' (and in various other programming languages as well).  This is a 'mini language' that handles all kinds of string operations.  I leave it to those who are curious about this to read documentation on it.  A tutorial is available at: https://docs.python.org/2/howto/regex.html

But for demonstration purposes, here is an example of using re to clean up the bedrooms variable:

In [4]:
#Reload the data file into a different dataframe to have a raw starting file
df2 = pd.read_csv('data/bay.csv')

# clean bedrooms using regular expressions
# see https://docs.python.org/2/howto/regex.html for a tutorial - it is a moderately advanced topic
for i in df2.bedrooms.index:
    r = re.search('(?<=\/ )(.*)(?=br)', df2.bedrooms[i])
    df2.bedrooms[i] = r.group(0) if r else np.nan
df2.bedrooms = df2.bedrooms.astype('float64')

df2['bedrooms'][:10]

0     1
1     1
2     1
3     1
4     3
5     2
6   NaN
7     2
8     1
9     1
Name: bedrooms, dtype: float64

Below is a concise version of the code to load the Bay Area rental data, load it into a DataFrame, and clean it up.

In [5]:
# import libraries and read in the csv file
import re as re, pandas as pd, numpy as np, requests, json
df = pd.read_csv('data/bay.csv')

# clean price and neighborhood
df.price = df.price.str.strip('$').astype('float64')
df.neighborhood = df.neighborhood.str.strip().str.strip('(').str.strip(')')

# break out the date into month day year columns
df['month'] = df['date'].str.split().str[0]
df['day'] = df['date'].str.split().str[1].astype('int32')
df['year'] = df['date'].str.split().str[2].astype('int32')
del df['date']

# clean bedrooms using regular expressions
# see https://docs.python.org/2/howto/regex.html for a tutorial - it is a moderately advanced topic
for i in df.bedrooms.index:
    r = re.search('(?<=\/ )(.*)(?=br)', df.bedrooms[i])
    df.bedrooms[i] = r.group(0) if r else np.nan
df.bedrooms = df.bedrooms.astype('float64')


# clean up the sqft using an iterator over the rows in the DataFrame
for i in df.sqft.index:
    if('ft' in df.sqft[i]):
        end = df.sqft[i].find('ft')
        begin = df.sqft[i].find('- ') + 2
        if(begin > end):
            begin = df.sqft[i].find('/ ') + 2
        df.sqft[i] = df.sqft[i][begin:end]
    else:
        df.sqft[i] = np.nan
df.sqft = df.sqft.astype('float64')


df.head()

,neighborhood,title,price,bedrooms,pid,longitude,subregion,link,latitude,sqft,month,day,year
0,bayview,Take A TOUR ON OUR ONE FURNISHED BEDROOM TODAY,950,1,4076905111,-122.396965,SF,/sfc/apa/4076905111.html,37.761216,NaN,Sep,18,2013
1,bayview,Only walking distance to major shopping centers.,950,1,4076901755,-122.396793,SF,/sfc/apa/4076901755.html,37.761080,NaN,Sep,18,2013
2,bayview,"furnished - 1 Bedroom(s), 1 Bath(s), Air Condi...",950,1,4076899340,-122.397100,SF,/sfc/apa/4076899340.html,37.762100,NaN,Sep,18,2013
3,financial district,"*NEW* Beautiful, Upscale Condo in Historic Jac...",3300,1,4067393707,-122.399747,SF,/sfc/apa/4067393707.html,37.798108,830,Sep,18,2013
4,visitacion valley,楼上全层3房,2000,3,4076901071,NaN,SF,/sfc/apa/4076901071.html,NaN,1280,Sep,18,2013


## Now we need to build a geocoding script.

We have latitude and longitude in our data, but want to get census geographies like block and tract and county.  How to do this?  We can use an FCC API to reverse geocode from lat, long to get these:

http://www.fcc.gov/developers/census-block-conversions-api

Test out the FCC API manually in your browser and look at how the results look.

Review the examples in Chapter 6 of Wes McKinney's book for retrieving data from a url.  I suggest using the JSON format in the API, but you can test either JSON or XML.

Also, since you will want to step through the DataFrame one row at a time, we use iterrows. 

http://pandas.pydata.org/pandas-docs/dev/generated/pandas.DataFrame.iterrows.html

**CAUTION: PLEASE DO NOT RUN THIS IN CLASS!!!**
Since the API is a SLOW for processing a lot of records, and works one record at a time, test it out with only a few records until you are positive you have it working correctly.  It may take several minutes to grind through all the records, and you don't want to get us locked out of the site.  You might want to print the row you are processing - maybe every tenth row number, to give you some sense of how it is progressing.

And watch out for those missing lat-longs.  You'll probably want to skip them since the API won't know what to do with them.

In [ ]:
#We use an FCC API to convert lat, long to census block and other geographies
#http://www.fcc.gov/developers/census-block-conversions-api
url = 'http://data.fcc.gov/api/block/2010/find?format=json&latitude='

# define the new geolocation fields for our dataframe
df['blockfips'] = ''
df['countyfips'] = ''
df['county'] = ''

#We need to iterate over the rows of the DataFrame and get data from the FCC API for each
#http://pandas.pydata.org/pandas-docs/dev/generated/pandas.DataFrame.iterrows.html
for i, row in df.iterrows():
    if row['latitude']>0: # and index<50:
        resp = requests.get(url+str(row['latitude'])+'&longitude='+str(row['longitude']))
        data = json.loads(resp.text)
        #print data
        df['countyfips'][i] = data['County']['FIPS']
        df['county'][i] = data['County']['name']
        df['blockfips'][i] = data['Block']['FIPS']
        if i%200 == 0: 
            print 'processing row: ', i

print df[:5]

Something you may find useful is that you can save files into a very efficient format called HDF5, which can store multiple tables in one file, and allow you to retrieve them easily.  Here is how to create a file and store the dataframe into it.  

Note: if you run this cell in class and not the preceding one, you won't have the geocoding results in it.  I've placed a copy of the resulting HDF5 file in the files section for easy access.

**Don't run this in class, either, or it will overwrite bay.h5**

In [8]:
store = pd.HDFStore('data/bay.h5')
store['rents'] = df
print store['rents'][:5]
store.close()

         neighborhood                                              title  \
0             bayview     Take A TOUR ON OUR ONE FURNISHED BEDROOM TODAY   
1             bayview   Only walking distance to major shopping centers.   
2             bayview  furnished - 1 Bedroom(s), 1 Bath(s), Air Condi...   
3  financial district  *NEW* Beautiful, Upscale Condo in Historic Jac...   
4   visitacion valley                                             楼上全层3房   

   price  bedrooms         pid   longitude subregion  \
0    950         1  4076905111 -122.396965        SF   
1    950         1  4076901755 -122.396793        SF   
2    950         1  4076899340 -122.397100        SF   
3   3300         1  4067393707 -122.399747        SF   
4   2000         3  4076901071         NaN        SF   

                       link   latitude  sqft month  day  year  \
0  /sfc/apa/4076905111.html  37.761216   NaN   Sep   18  2013   
1  /sfc/apa/4076901755.html  37.761080   NaN   Sep   18  2013   
2  /sfc/apa

C:\Anaconda\lib\site-packages\pandas\io\pytables.py:2453: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['neighborhood', 'title', 'subregion', 'link', 'month', 'blockfips', 'countyfips', 'county']]

  warnings.warn(ws, PerformanceWarning)


And here is how to reload it to a dataframe from the HDF5 file.

In [9]:
store = pd.HDFStore('data/bay.h5')
df = store['rents']
print df['blockfips'][:10]

0    060750227022008
1    060750227022008
2    060750227022005
3    060750105002005
4                   
5                   
6                   
7                   
8    060750227022005
9    060750227021002
Name: blockfips, dtype: object


One more new trick for today: extracting a substring using the lambda function, to extract tractfips from blockfips.

A quick explanation from: http://www.python-course.eu/lambda.php:

The lambda operator or lambda function is a way to create small anonymous functions, i.e. functions without a name. These functions are throw-away functions, i.e. they are just needed where they have been created. Lambda functions are mainly used in combination with the functions filter(), map() and reduce().

The general syntax of a lambda function is quite simple:
lambda argument_list: expression 
The argument list consists of a comma separated list of arguments and the expression is an arithmetic expression using these arguments. You can assign the function to a variable to give it a name. 
The following example of a lambda function returns the sum of its two arguments:

In [10]:
f = lambda x, y : x + y
f(1,1)

2

Let's use this approach to extract the census tract values, which are the first 10 characters in the fipsblock values.

In [11]:
df['tractfips'] = df['blockfips'].map(lambda x: x[:11])
df['tractfips'][:10]

0    06075022702
1    06075022702
2    06075022702
3    06075010500
4               
5               
6               
7               
8    06075022702
9    06075022702
Name: tractfips, dtype: object

And here we write our dataframe to a csv file.

In [12]:
df.to_csv('Data/rents.csv')

We wrap up with a bit of exploring the data.  Computing means of sqft by county.

In [13]:
df.groupby('county')['sqft'].mean()

county
                 1149.289189
Alameda          1188.153439
Butte            1350.000000
Calaveras                NaN
Contra Costa     1389.156716
Humboldt         1952.000000
Lake             1487.157895
Marin            1401.006623
Mendocino        1225.000000
Merced            850.000000
Monterey         3773.000000
Napa             1543.306122
Placer           2700.000000
Sacramento       1000.000000
San Benito       1850.000000
San Francisco    1082.411960
San Joaquin      2154.875000
San Mateo        1094.573604
Santa Clara      1104.003643
Siskiyou          750.000000
Solano           1236.367925
Sonoma           1260.573248
Name: sqft, dtype: float64

Notice that we have extraneous counties in the data. Some listings advertised in the Bay Area Craigslist but were for properties in other counties. Let's eliminate the listings that are outside the Bay Area.

In [14]:
bayarea = ['Alameda','Contra Costa','Marin','Napa','San Francisco','San Mateo','Santa Clara','Solano','Sonoma']
df_bay = df[df['county'].isin(bayarea)]
df_bay.groupby('county')['price'].count()

county
Alameda          295
Contra Costa     174
Marin            213
Napa              71
San Francisco    692
San Mateo        529
Santa Clara      690
Solano           143
Sonoma           266
Name: price, dtype: int64

In [15]:
df_bay.groupby('county')['sqft'].mean()

county
Alameda          1188.153439
Contra Costa     1389.156716
Marin            1401.006623
Napa             1543.306122
San Francisco    1082.411960
San Mateo        1094.573604
Santa Clara      1104.003643
Solano           1236.367925
Sonoma           1260.573248
Name: sqft, dtype: float64